In [1]:
!pip install pyxlsb

README: 
Objectif récuppérer les loyers du parcs privé par communes et récuppérer les contours géographiques des communes. On exporte ensuite une nouvelle base de données. 

In [2]:
import pandas as pd
import pyxlsb
import geopandas as gpd


In [3]:
df = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")

In [4]:
df

,id_zone,INSEE,LIBGEO,DEP,REG,EPCI,TYPPRED,loypredm2,lwr.IPm2,upr.IPm2,R2adj,NBobs_maille,NBobs_commune
0,2362,1001,L'Abergement-Clémenciat,1,84,200069193,maille,"9,372335348","7,409663525","11,85487972","0,774249278",701,2
1,2010,1002,L'Abergement-de-Varey,1,84,240100883,maille,"8,63555202","6,72107751","11,09535764","0,719598662",861,0
2,1456,1004,Ambérieu-en-Bugey,1,84,240100883,commune,"10,07450708","7,865307059","12,90422511","0,670160882",2156,2156
3,2362,1005,Ambérieux-en-Dombes,1,84,200042497,maille,"9,372335348","7,409663525","11,85487972","0,774249278",701,81
4,2055,1006,Ambléon,1,84,200040350,maille,"8,96695486","7,101593346","11,3222872","0,752823051",842,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35436,1802,97420,Sainte-Suzanne,974,4,249740119,commune,"10,67356968","8,791530189","12,95850521","0,788143839",847,411
35437,1802,97421,Salazie,974,4,249740093,maille,"10,38745938","8,545191709","12,62690364","0,788143839",847,11
35438,1608,97422,Le Tampon,974,4,249740085,commune,"10,81548488","8,696332802","13,45103918","0,81541405",3237,3237
35439,1607,97423,Les Trois-Bassins,974,4,249740101,maille,"12,34315971","8,597802405","17,72006199","0,460731322",1669,90


On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2

In [5]:
df_private = df[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes. 

In [6]:
df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)
df_private = gpd.GeoDataFrame(df_private)


/tmp/ipykernel_4905/3783642682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_private.rename(columns = {'INSEE':'codgeo'}, inplace = True)


In [7]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base). 
"""
for i in range(0,df_private.shape[0]):
    if  len(df_private["codgeo"][i]) == 4: 
            df_private["codgeo"][i] = "0" +  df_private["codgeo"][i]


In [8]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

In [9]:
df_private

,LIBGEO,upr.IPm2
codgeo,,
01001,L'Abergement-Clémenciat,"11,85487972"
01002,L'Abergement-de-Varey,"11,09535764"
01004,Ambérieu-en-Bugey,"12,90422511"
01005,Ambérieux-en-Dombes,"11,85487972"
01006,Ambléon,"11,3222872"
...,...,...
97420,Sainte-Suzanne,"12,95850521"
97421,Salazie,"12,62690364"
97422,Le Tampon,"13,45103918"


On récuppère les contours géographique des communes.

In [10]:
gdf = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


In [11]:
gdf

,ID,INSEE_COM,NOM,NOM_M,STATUT,INSEE_REG,INSEE_DEP,INSEE_CAN,INSEE_ARR,SIREN_EPCI,POPULATION,TYPE,geometry
0,COMMUNE_0000000009726955,59668,Zuydcoote,ZUYDCOOTE,Commune simple,32,59,17,4,245900428,1601,COM,"POLYGON ((2.49286 51.07342, 2.49271 51.07340, ..."
1,COMMUNE_0000000009726956,59340,Leffrinckoucke,LEFFRINCKOUCKE,Commune simple,32,59,17,4,245900428,4207,COM,"POLYGON ((2.46625 51.06674, 2.46581 51.06663, ..."
2,COMMUNE_0000000009726957,59260,Ghyvelde,GHYVELDE,Commune simple,32,59,17,4,245900428,4131,COM,"POLYGON ((2.52222 51.01007, 2.52267 51.00923, ..."
3,COMMUNE_0000000009726958,59107,Bray-Dunes,BRAY-DUNES,Commune simple,32,59,17,4,245900428,4495,COM,"POLYGON ((2.50693 51.06050, 2.50932 51.06075, ..."
4,COMMUNE_0000000009726959,59605,Uxem,UXEM,Commune simple,32,59,12,4,200040954,1443,COM,"POLYGON ((2.51650 51.01992, 2.51604 51.01977, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34992,ARR_MUNI0000000009761161,13203,Marseille 3e Arrondissement,MARSEILLE 3E ARRONDISSEMENT,Préfecture de région,93,13,98,3,200054807,51792,ARM,"POLYGON ((5.39313 43.31640, 5.39292 43.31637, ..."
34993,ARR_MUNI0000000009761162,13202,Marseille 2e Arrondissement,MARSEILLE 2E ARRONDISSEMENT,Préfecture de région,93,13,98,3,200054807,25217,ARM,"MULTIPOLYGON (((5.37474 43.30156, 5.37464 43.3..."
34994,ARR_MUNI0000000009761163,13201,Marseille 1er Arrondissement,MARSEILLE 1ER ARRONDISSEMENT,Préfecture de région,93,13,98,3,200054807,39893,ARM,"POLYGON ((5.37226 43.29097, 5.37229 43.29101, ..."
34995,ARR_MUNI0000000009761407,13209,Marseille 9e Arrondissement,MARSEILLE 9E ARRONDISSEMENT,Préfecture de région,93,13,98,3,200054807,76633,ARM,"MULTIPOLYGON (((5.45515 43.21166, 5.45517 43.2..."


On ne garde que le code INSEE et les données polygoniales. 

In [12]:
gdf.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)

In [13]:
gdf = gdf[["codgeo", "geometry"]]

In [14]:
gdf = gdf.set_index('codgeo')

On joint nos deux base à partir de codgeo. 

In [15]:
gdf_privated = pd.concat([df_private,gdf], axis = 1, join = "inner")

In [16]:
gdf_privated

,LIBGEO,upr.IPm2,geometry
codgeo,,,
01001,L'Abergement-Clémenciat,"11,85487972","POLYGON ((4.95825 46.15334, 4.95810 46.15342, ..."
01002,L'Abergement-de-Varey,"11,09535764","POLYGON ((5.43369 45.98393, 5.43441 45.98428, ..."
01004,Ambérieu-en-Bugey,"12,90422511","POLYGON ((5.40942 45.94592, 5.40949 45.94718, ..."
01005,Ambérieux-en-Dombes,"11,85487972","POLYGON ((4.94292 45.97974, 4.94275 45.97998, ..."
01006,Ambléon,"11,3222872","POLYGON ((5.57088 45.75316, 5.57187 45.75200, ..."
...,...,...,...
97420,Sainte-Suzanne,"12,95850521","POLYGON ((55.57407 -20.99343, 55.57408 -20.993..."
97421,Salazie,"12,62690364","POLYGON ((55.54066 -21.00411, 55.54064 -21.004..."
97422,Le Tampon,"13,45103918","POLYGON ((55.66050 -21.20333, 55.65868 -21.201..."


In [17]:
gdf_privated.to_csv(r'loyer_privé.csv')